importing some preliminary libraries

In [3]:
import numpy as np
from numpy.linalg import norm
 
from random import normalvariate
from math import sqrt

we are trying to implement SVD for a given matrix from scratch.
Any matrix A can be broken down into a product of two Untitary matrices and a diagonal matrix.
$$A = U\Sigma V^{T} -(1)$$ 
here $U$ is the left singular matrix $\Sigma$ is a diagonal matrix and $V$ is the right singular matrix

here,

$V = \bigl(\begin{smallmatrix}
v_{1} &.  &.  &.  &.  &v_{n} 
\end{smallmatrix}\bigr)$ is the list of right singular vectors. They form an orthonormal basis for the input vector space.

$U = \bigl(\begin{smallmatrix}
u_{1} &.  &.  &.  &.  &u_{n} 
\end{smallmatrix}\bigr)$ is the list of left singular vectors. They form an orthonormal basis for output vector space

and $\Sigma$ is a diagonal matrix consisting of Singular Values in the diagonal

$\Sigma = \begin{pmatrix}
 &\sigma _{1} & 0 &0  &0  &0 \\ 
 &0 &\sigma _{2}  &0  &0  &0 \\ 
 &0  &0  &\sigma _{3}  &0  &0 \\ 
 &0  &0  &0  & \sigma _{4} &0 \\ 
 &0 &0  &0  &0  &\sigma _{5} \\ 
\end{pmatrix}$

we can reframe $(1)$ as $AV = U\Sigma$ which inturn implies $Av_{n} = \sigma_{n}u_{n} -(2)$
it can also be written as $A = \sum_{i}\sigma_{i}u_{i}v_{i}^T - (3)$

$(2)$ is similiar to an Eigenvalue equation but $v_{n}$ and $u_{n}$ lives in different vector spaces.

We are trying to find out $v_{n}$ when a matrix A is given.

$v_{n}$ will be eigenvectors of this matrix $B = A^{T}A$

$A^{T}A = A = (U\Sigma V^{T})^{T}U\Sigma V^{T} = V\Sigma U^{T}U\Sigma V^{T} = V\Sigma^{2} V^{T}$

we can find $v_{n}$ by a method called power method.

in this method we start with a random unit vector $x$

then we apply $lim_{s\rightarrow \infty }B^sx$  which will be in the span of $v_{n}$

we can see this by expressing $x$ in the basis of $v_{1},...,v_{n}$

$$x = \sum_{i} c_{i}v_{i}$$

$$B^{s}x = \sum_{i} c_{i}B^{s}v_{i} = \sum_{i} c_{i}\sigma_{i}^{2s}v_{i}$$

if $\sigma_{1} > \sigma_{2}, \sigma_{1}^{2s} >>>> \sigma_{2}^{2s}$ and $B^{s}x \approx c_{1}\sigma_{1}^{2s}v_{1}$ and so  $B^{s}x$ will lie in the span of $v_{1}$ 

we can obtain $v_{i}$ from $x$ by normalising it 




Input a matrix A

In [13]:
n_rows = int(input("Enter number of rows: "))
n_cols = int(input("Enter number of columns: "))

B = [[int(input("Enter value for {}. row and {}. column: ".format(r + 1, c + 1))) for c in range(n_cols)] for r in range(n_rows)]
A = np.array(B).astype('float64')

Enter number of rows:  2
Enter number of columns:  2
Enter value for 1. row and 1. column:  1
Enter value for 1. row and 2. column:  2
Enter value for 2. row and 1. column:  3
Enter value for 2. row and 2. column:  4


Here we define a function randomUnitVector(n) that returns a random unit vector of dimension n, $x$

In [6]:
def randomUnitVector(n):
    unnormalized = [normalvariate(0, 1) for _ in range(n)]
    theNorm = sqrt(sum(x * x for x in unnormalized))
    return [x / theNorm for x in unnormalized]

Here we define a function svd_1d(A, epsilon=1e-10), that returns $v_{1}$ if given a matrix $A$ as its input by recursivly applying B to x and stopping when $\left | B^{s}x - B^{s-1}x \right | < epsilon$

In [7]:
def svd_1d(A, epsilon=1e-10):
    ''' The one-dimensional SVD '''
 
    n, m = A.shape
    x = randomUnitVector(m)
    lastV = None
    currentV = x
    B = np.dot(A.T, A)
 
    iterations = 0
    while True:
        iterations += 1
        lastV = currentV
        currentV = np.dot(B, lastV)
        currentV = currentV / norm(currentV) #normalise
 
        if abs(np.dot(currentV, lastV)) > 1 - epsilon: #checks if currentV is close enough to the first singular vector
            print("converged in {} iterations!".format(iterations))
            return currentV

In [8]:
print("v1 = {}.".format(svd_1d(A, epsilon=1e-10)))

converged in 3 iterations!
v1 = [0.57604844 0.81741556].


we can find the other $v_{2}$ by subtracting $\sigma_{1}u_{1}v_{1}^T$ from A , $A' = A -\sigma_{1}u_{1}v_{1}^T$ and passing $A'$ to svd_1d().

all other singular vectors can be found this way by recursivly subtracting rank 1 components of the previous singular vector from A and then using power method

In [9]:
def svd(A, epsilon=1e-10):
    n, m = A.shape
    svdSoFar = []
 
    for i in range(m):
        matrixFor1D = A.copy()
 
        for singularValue, u, v in svdSoFar[:i]:
            matrixFor1D -= singularValue * np.outer(u, v)
 
        v = svd_1d(matrixFor1D, epsilon=epsilon)  # next singular vector
        u_unnormalized = np.dot(A, v)
        sigma = norm(u_unnormalized)  # next singular value
        u = u_unnormalized / sigma
 
        svdSoFar.append((sigma, u, v))
 
    # transform it into matrices of the right shape
    singularValues, us, vs = [np.array(x) for x in zip(*svdSoFar)]
 
    return singularValues, us.T, vs

In [16]:
theSVD = svd(A)

print(theSVD[0]) #list of singular values
print(theSVD[1]) #left singular matrix
print(theSVD[2]) #left singular matrix

converged in 4 iterations!
converged in 2 iterations!
[5.4649857  0.36596619]
[[-0.40455358  0.9145143 ]
 [-0.9145143  -0.40455357]]
[[-0.57604844 -0.81741556]
 [-0.81741556  0.57604844]]
